# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [12]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [13]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("mapbox")
# Set the Mapbox API
px.set_mapbox_access_token(mapbox_token)

# Import Data

In [14]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [15]:
# Define Panel Visualization Functions
def housing_units_per_year():
    "Housing Units Per Year."
    mean_housing_units = sfo_data.groupby(["year"])["housing_units"].mean()
    sfo_grouped_min = sfo_data.groupby(["year"])["housing_units"].min().min()
    sfo_grouped_max = sfo_data.groupby(["year"])["housing_units"].max().max()
    sfo_grouped_std = sfo_data["housing_units"].std()
    #TODO fix years...
    first_year = mean_housing_units.index[0]
    last_year = mean_housing_units.index[-1]
    fig_housing_units = px.bar(mean_housing_units,
                               title = "Housing Units in San Francisco from 2010 to 2016")
    return fig_housing_units
def average_gross_rent():
    "Average Gross Rent in San Francisco Per Year."
    sfo_grouped_by_year = sfo_data.groupby(["year"])
    mean_gross_rent = sfo_grouped_by_year["gross_rent"].mean()
    mean_gross_rent_plot = px.bar(mean_gross_rent,
                                  title = "Average Gross Rent in San Francisco")
    return mean_gross_rent_plot
def average_sales_price():
    "Average Sales Price Per Year."
    sfo_grouped_by_year = sfo_data.groupby(["year"])
    mean_price_per_sqft = sfo_grouped_by_year["sale_price_sqr_foot"].mean()
    mean_price_per_sqft_plot = px.line(mean_price_per_sqft,
                                       title = "Average Sales Price per Year in San Francisco")
    return mean_price_per_sqft_plot
def average_price_by_neighborhood():
    "Average Prices by Neighborhood."
    sfo_neighborhood = pd.read_csv("Data/sfo_neighborhoods_census_data.csv")
    sfo_grouped_by_year_and_nbhd = sfo_neighborhood.groupby(["year", "neighborhood"])
    sfo_grouped_by_year_and_nbhd.head()
    avg_price_mean = sfo_grouped_by_year_and_nbhd.mean()
    sfo_grouped_year_and_nbhd_plot = avg_price_mean.hvplot.line(
        x='year',
        y='sale_price_sqr_foot',
        xlabel='Year',
        ylabel='Price per Sqr Foot',
        groupby='neighborhood',
        width=600
        ).opts(yformatter='%0f')
    return sfo_grouped_year_and_nbhd_plot
def top_most_expensive_neighborhoods():
    "Top 10 Most Expensive Neighborhoods."
    groupby_neighborhood = sfo_data.groupby(["neighborhood"]).mean()
    top_10_most_expensive = groupby_neighborhood.sort_values(by="sale_price_sqr_foot", ascending=False).head(10)
    top_10_most_expensive = top_10_most_expensive.reset_index()
    top_10_most_expensive_plot = top_10_most_expensive.hvplot.bar(
        x="neighborhood",
        y="sale_price_sqr_foot",
        xlabel = "Neighborhoods",
        ylabel = "Average Sale Price Per Square Foot",
        title="Top 10 Most Expensive Neighborhoods in SFO",
        height=400,
        rot=90)
    return top_10_most_expensive_plot
def parallel_coordinates():
    "Parallel Coordinates Plot."
    groupby_neighborhood = sfo_data.groupby(["neighborhood"]).mean()
    top_10_most_expensive = groupby_neighborhood.sort_values(by="sale_price_sqr_foot", ascending=False).head(10)
    top_10_most_expensive = top_10_most_expensive.reset_index()
    top_10_most_expensive_parallel = px.parallel_coordinates(top_10_most_expensive, color='sale_price_sqr_foot')
    return top_10_most_expensive_parallel
def parallel_categories():
    "Parallel Categories Plot."
    groupby_neighborhood = sfo_data.groupby(["neighborhood"]).mean()
    top_10_most_expensive = groupby_neighborhood.sort_values(by="sale_price_sqr_foot", ascending=False).head(10)
    top_10_most_expensive = top_10_most_expensive.reset_index()
    top_10_most_expensive_categories =  px.parallel_categories(
            top_10_most_expensive,
            dimensions=["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"],
            color="sale_price_sqr_foot",
            color_continuous_scale=px.colors.sequential.Inferno,
        )
    return top_10_most_expensive_categories
def neighborhood_map():
    "Neighborhood Map"
    mean_data = sfo_data.groupby(['neighborhood']).mean()
    mean_data.reset_index(inplace=True)
    combined_data = pd.concat([df_neighborhood_locations, mean_data], axis='columns', join='inner')
    combined_data = combined_data.iloc[:, 1:]
    px.set_mapbox_access_token(mapbox_token)
    neighborhood_map = px.scatter_mapbox(
            combined_data,
            lat="Lat",
            lon="Lon",
            size="gross_rent",
            color="neighborhood"
        )
    return neighborhood_map

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [16]:

map_plt = pn.Column(neighborhood_map())
yearly_market_analysis = pn.Column(housing_units_per_year, average_gross_rent, average_sales_price())
neighborhood_analysis = pn.Column(average_price_by_neighborhood, top_most_expensive_neighborhoods())
parallel_plot_analysis = pn.Column(parallel_categories(), parallel_coordinates())
panel_content = pn.Tabs(
                        ('Welcome', map_plt),
                        ('Yearly Market Analysis', yearly_market_analysis),
                        ('Neighborhood Analysis', neighborhood_analysis),
                        ('Parallel Plot Analysis', parallel_plot_analysis)
)
panel_content


Tabs
    [0] Column
        [0] Plotly(Figure)
    [1] Column
        [0] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive02190')
        [1] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive02196')
        [2] Plotly(Figure)
    [2] Column
        [0] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive02281')
                    [0] HoloViews(DynamicMap, name='interactive02281')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                        [1] VSpacer()
        [1] HoloViews(Bars)
    [3] Column
        [0] Plotly(Figure)
        [1] Plotly(Figure)

## Serve the Panel Dashboard

In [17]:
panel_content.servable()

Tabs
    [0] Column
        [0] Plotly(Figure)
    [1] Column
        [0] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive02190', viewport={'xaxis.range': [2009.5, ...})
        [1] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive02196', viewport={'xaxis.range': [2009.5, ...})
        [2] Plotly(Figure, viewport={'xaxis.range': [2010, ...})
    [2] Column
        [0] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive02281')
                    [0] HoloViews(DynamicMap, name='interactive02281')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                        [1] VSpacer()
        [1] HoloViews(Bars)
    [3] Column
        [0] Plotly(Figure)
        [1] Plotly(Figure)